# DuckDB Python Integration

## Introduction

## Learning Objectives

## Sample Datasets

## Installation and Setup

### Library Import and Initial Setup

## Installing and Loading Extensions

## Reading Data from Multiple Sources

### Verifying the Connection

### Reading CSV Files from URLs

### Understanding DuckDB Relations

## Seamless Integration with Pandas DataFrames

### Querying DataFrames with SQL

### The Bidirectional Workflow

### Performance Considerations

## Polars Interoperability

## Result Conversion and Output Formats

### Converting to Python Objects

### Converting to Pandas DataFrames

### Converting to NumPy Arrays

### Converting to Apache Arrow Tables

### Choosing the Right Format

## Writing Data To Disk

## Persistent Storage and Database Files

### Creating a Persistent Database

### Connecting to Existing Databases

### Connection Management and Cleanup

### Use Cases for Persistent vs In-Memory Databases

## Prepared Statements and Parameters

## Key Takeaways

## Exercises

### Exercise 1: Installation and Basic Queries

### Exercise 2: Loading Remote Data

### Exercise 3: SQL to DataFrame Conversion

### Exercise 4: Querying DataFrames with SQL

### Exercise 5: Bidirectional Workflows

### Exercise 6: Result Format Conversion

### Exercise 7: Persistent Storage

### Exercise 8: Joining SQL and Python Logic

### Exercise 9: Writing Results to Files

### Exercise 10: Practical Integration Challenge